# Scratch Pad

In [1]:
import re
import etl
import math
import pandas as pd

from copy import copy

## Reference Files

In [2]:
MDRM_FILE = '/Users/admin/notebooks/fdic_data/financial_data_raw/mdrm_codes'
FDIC_FILE = '/Users/admin/Downloads/FFIEC CDR Call Bulk Subset of Schedules 2017/FFIEC CDR Call Subset of Schedules 2017(1 of 2).txt'
FDIC_FILE2 = '/Users/admin/Downloads/FFIEC CDR Call Bulk Subset of Schedules 2017/FFIEC CDR Call Subset of Schedules 2017(2 of 2).txt'

## Connect to Mongo Instance for Prototyping

In [3]:
import pymongo as mongo

In [4]:
client = mongo.MongoClient()

In [5]:
client.database_names()

['admin', 'ffeic_bank', 'local']

In [6]:
db = client['ffeic_bank']

## Load ETL Instances

In [7]:
mdrm_etl = etl.MDRMInfoETL(MDRM_FILE)

In [8]:
fdic_etl = etl.FDICDataETL(FDIC_FILE, [10057, 3510])

In [9]:
company_etl = etl.FDICCompanyInfoETL(FDIC_FILE, [10057, 3510])

###### Drop a Collection

In [10]:
# db.drop_collection('financial_data')

In [11]:
# db.drop_collection('ffeic_reporting')

In [12]:
# db.drop_collection('company_info')

In [13]:
# db.drop_collection('mdrm_details')

###### Load Initial Data

In [14]:
ffeic_coll = db['ffeic_reporting']
comp_coll = db['company_info']
mdrm_coll = db['mdrm_details']

In [15]:
fdic_trans = fdic_etl.transform()

In [16]:
fdic_etl.load(ffeic_coll, fdic_trans)

In [17]:
newfdic_etl = etl.FDICDataETL(FDIC_FILE2, [10057, 3510])

In [18]:
newfdic_trans = newfdic_etl.transform()

In [19]:
newfdic_etl.load(ffeic_coll, newfdic_trans)

In [20]:
mdrm_trans = mdrm_etl.transform()

In [21]:
mdrm_etl.load(mdrm_coll, mdrm_trans)

In [22]:
comp_trans = company_etl.transform()

In [23]:
company_etl.load(comp_coll, comp_trans)

In [24]:
db.collection_names()

['trial', 'ffeic_reporting', 'mdrm_details', 'company_info']

In [25]:
for item in comp_coll.find():
    print(item)

{'_id': ObjectId('5945f6bedbc55442038c69ca'), 'record_updated': '2017-03-31', 'financial_institution_name': 'BANK OF HANCOCK COUNTY', 'fdic_certificate_number': 10057, 'location': {'financial_institution_address': '12855 BROAD STREET ', 'financial_institution_city': 'SPARTA', 'financial_institution_state': 'GA', 'financial_institution_zip_code': 31087}, 'identifiers': {'idrssd': 37, 'occ_charter_number': 0, 'ots_docket_number': 16553, 'primary_aba_routing_number': 61107146, 'financial_institution_filing_type': 51}}
{'_id': ObjectId('5945f6bedbc55442038c69cb'), 'record_updated': '2017-03-31', 'financial_institution_name': 'BANK OF AMERICA, NATIONAL ASSOCIATION', 'fdic_certificate_number': 3510, 'location': {'financial_institution_address': '100 NORTH TRYON STREET, SUITE 170 ', 'financial_institution_city': 'CHARLOTTE', 'financial_institution_state': 'NC', 'financial_institution_zip_code': 28202}, 'identifiers': {'idrssd': 480228, 'occ_charter_number': 13044, 'ots_docket_number': 0, 'pri

In [26]:
for item in ffeic_coll.find():
    print(item)

{'_id': ObjectId('5945f6bcdbc55442038c66bf'), 'reporting_period_end_date': '2017-03-31', 'financial_institution_name': 'BANK OF HANCOCK COUNTY', 'fdic_certificate_number': 10057, 'last_datetime_submission_updated_on': '2017-04-24T12:00:12', 'financials': {'RCFD0071': nan, 'RCFD0081': nan, 'RCFD0426': nan, 'RCFD1248': nan, 'RCFD1249': nan, 'RCFD1250': nan, 'RCFD1251': nan, 'RCFD1252': nan, 'RCFD1253': nan, 'RCFD1254': nan, 'RCFD1255': nan, 'RCFD1256': nan, 'RCFD1403': nan, 'RCFD1406': nan, 'RCFD1407': nan, 'RCFD1583': nan, 'RCFD1594': nan, 'RCFD1597': nan, 'RCFD1754': nan, 'RCFD1773': nan, 'RCFD2130': nan, 'RCFD2145': nan, 'RCFD2150': nan, 'RCFD2160': nan, 'RCFD2170': nan, 'RCFD2930': nan, 'RCFD2948': nan, 'RCFD3000': nan, 'RCFD3123': nan, 'RCFD3163': nan, 'RCFD3190': nan, 'RCFD3200': nan, 'RCFD3210': nan, 'RCFD3230': nan, 'RCFD3300': nan, 'RCFD3505': nan, 'RCFD3506': nan, 'RCFD3507': nan, 'RCFD3529': nan, 'RCFD3530': nan, 'RCFD3545': nan, 'RCFD3548': nan, 'RCFD3632': nan, 'RCFD3656': n

In [ ]:
db.drop_collection('ffeic_reporting')

In [ ]:
db.drop_collection('mdrm_details')

In [ ]:
db.drop_collection('company_info')

# Prototyping

In [ ]:
dctold = {'id': 1, 'date': '2017-01-01', 'sub':{'a': 1, 'b': float('nan'), 'c': 7}}
dctnew = {'id': 1, 'date': '2017-01-01', 'sub':{'a': float('nan'), 'b': 5, 'c': 3}}

In [ ]:
def prep_update(dctold, dctnew):
    for key, value in dctnew.items():
        if isinstance(value, dict):
            prep_update(dctold[key], value)
        elif value and not (isinstance(value, (int, float)) and math.isnan(value)):
            dctold[key] = value

In [ ]:
cop = dctold.copy()

In [ ]:
prep_update(cop, dctnew)

In [ ]:
cop